## 3. Collaborative Filtering (협업 필터링 : 사용자 리뷰 기반)

나 : Up, 주토피아

다른 사람 : Up, 주토피아, 인사이드 아웃

In [3]:
import surprise # 추천 시스템 패키지 / anaconda prompt를 관리자 권한으로 실행한 뒤 $ pip install scikit-surprise
surprise.__version__

'1.1.1'

In [4]:
import pandas as pd
from surprise import Reader, Dataset, SVD # Reader class
from surprise.model_selection import cross_validate

In [5]:
ratings = pd.read_csv('ratings_small.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [6]:
ratings['rating'].min()

0.5

In [7]:
ratings['rating'].max()

5.0

In [8]:
reader = Reader(rating_scale=(0.5, 5)) # 디폴트가 1~5 까지인데 우리 파일에는 최하 평점이 0.5 이므로

In [10]:
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader=reader) # 반드시 이렇게 3개만 (라이브러리에서 그렇게 제한해서)
data

In [11]:
# 아마 영화 추천 시스템.ipynb 에서 배움
svd = SVD(random_state=0)

In [12]:
# cross_validate : 모델을 평가할 때 데이터를 여러번 나눠서 교차로 검증
# cv=5 : 5세트로 나눔
cross_validate(svd, data, measures=['RMSE', 'MAE'], cv=5, verbose=True) # root min square, min absolute error

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9001  0.8968  0.8943  0.8962  0.8950  0.8965  0.0020  
MAE (testset)     0.6942  0.6890  0.6883  0.6872  0.6895  0.6896  0.0024  
Fit time          3.50    3.37    4.03    3.60    4.51    3.80    0.42    
Test time         0.09    0.19    0.08    0.12    0.18    0.13    0.05    


{'test_rmse': array([0.90010683, 0.89683236, 0.89431302, 0.8961704 , 0.89504893]),
 'test_mae': array([0.69420149, 0.68898591, 0.68829465, 0.6872162 , 0.68951921]),
 'fit_time': (3.50003719329834,
  3.366678476333618,
  4.029540538787842,
  3.5951991081237793,
  4.513662815093994),
 'test_time': (0.08659529685974121,
  0.19404101371765137,
  0.0799722671508789,
  0.12221574783325195,
  0.18076395988464355)}

교차 검증이란? (K-Fold 교차 검증)

ex) 100 개 데이터

cv가 5이므로

A: 백개의 데이터 중 1-20 
B: 21-40
C: 41-60
D:61-80
E:81-100

이렇게 5개의 세트로 나눔

ABCD (train set) E (test set) 
    --> A~D 로 훈련을 시키고 E로 테스트 
ABCE (train set) D (test set) 
ABDE (train set) C (test set) 
ACDE (train set) B (test set) 
BCDE(train set) A (test set) 

svd 모델 : 어떤 사용자가 영화들을 평가했을 때 새로운 영화 데이터가 제공되면 과연 그 영화를 몇 점으로 평가할 것인가를 예측

In [13]:
trainset = data.build_full_trainset()
svd.fit(trainset)

In [14]:
ratings[ratings['userId']== 1] # 이 사람은 20개의 데이터에 대해 평가를 했음

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205
5,1,1263,2.0,1260759151
6,1,1287,2.0,1260759187
7,1,1293,2.0,1260759148
8,1,1339,3.5,1260759125
9,1,1343,2.0,1260759131


In [15]:
# 위 사람이 id 가 302인 영화를 어떻게 평가할지 예측
svd.predict(1, 302) # (userId, movieId) / r_ui : 실제로 해당 user가 평가한 평점 / 세번째 인자에 임의로 3을 넣으면 r_ui가 3이됨
#svd.predict(1, 302, 3)

Prediction(uid=1, iid=302, r_ui=None, est=2.7142061734434044, details={'was_impossible': False})

In [17]:
svd.predict(1, 1029, 3) # UserId = 1 번인 사람이 Movie Id = 1029 인 영화에 대해서 실제 평가가 3점일 때, 예측 평가 점수?

Prediction(uid=1, iid=1029, r_ui=3, est=2.8814455446761933, details={'was_impossible': False})

In [18]:
ratings[ratings['userId']==100]

,userId,movieId,rating,timestamp
15273,100,1,4.0,854193977
15274,100,3,4.0,854194024
15275,100,6,3.0,854194023
15276,100,7,3.0,854194024
15277,100,25,4.0,854193977
15278,100,32,5.0,854193977
15279,100,52,3.0,854194056
15280,100,62,3.0,854193977
15281,100,86,3.0,854194208
15282,100,88,2.0,854194208


In [19]:
svd.predict(100, 1029)

Prediction(uid=100, iid=1029, r_ui=None, est=3.7705476478414846, details={'was_impossible': False})